In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/Opción I')
print(os.getcwd())

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1lBhm112YiMUZzXUG1wX-mKCH1zi7fLPe/Opción I


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
data = pd.read_csv('procesado.csv')
data.head()

,Unnamed: 0,Z_LOC,CALI,RMED,RDEP,RHOB,GR,NPHI,DTC,SP,BS,ROP,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY
0,0,-469.501831,19.480835,1.611410,1.798681,1.884186,80.200851,0.382279,161.131180,24.612379,12.2706,34.636410,-0.574928,Shale
1,1,-469.653809,19.468800,1.618070,1.795641,1.889794,79.262886,0.382279,160.603470,23.895531,12.2706,34.636410,-0.570188,Shale
2,2,-469.805786,19.468800,1.626459,1.800733,1.896523,74.821999,0.382279,160.173615,23.916357,12.2706,34.779556,-0.574245,Shale
3,3,-469.957794,19.459282,1.621594,1.801517,1.891913,72.878922,0.382279,160.149429,23.793688,12.2706,39.965164,-0.586315,Shale
4,4,-470.109772,19.453100,1.602679,1.795299,1.880034,71.729141,0.382279,160.128342,24.104078,12.2706,57.483765,-0.597914,Shale


In [ ]:
del data['Unnamed: 0']
data.head()

,Z_LOC,CALI,RMED,RDEP,RHOB,GR,NPHI,DTC,SP,BS,ROP,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY
0,-469.501831,19.480835,1.611410,1.798681,1.884186,80.200851,0.382279,161.131180,24.612379,12.2706,34.636410,-0.574928,Shale
1,-469.653809,19.468800,1.618070,1.795641,1.889794,79.262886,0.382279,160.603470,23.895531,12.2706,34.636410,-0.570188,Shale
2,-469.805786,19.468800,1.626459,1.800733,1.896523,74.821999,0.382279,160.173615,23.916357,12.2706,34.779556,-0.574245,Shale
3,-469.957794,19.459282,1.621594,1.801517,1.891913,72.878922,0.382279,160.149429,23.793688,12.2706,39.965164,-0.586315,Shale
4,-470.109772,19.453100,1.602679,1.795299,1.880034,71.729141,0.382279,160.128342,24.104078,12.2706,57.483765,-0.597914,Shale


In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import *

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
data = shuffle(data)
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
est = DecisionTreeClassifier(max_depth=17) ##Se usará la mejor profundidad encontrada
est.fit(X_train,y_train)
features = est.feature_importances_
features.argsort()[::-1]

array([ 6, 10,  9,  8,  0,  2,  5,  7,  4,  3,  1, 11])

In [ ]:
bestf = features.argsort()[::-1][:5]
print(bestf)

X_train_bf = X_train.iloc[:,bestf]
X_test_bf = X_test.iloc[:,bestf]

[ 6 10  9  8  0]


In [ ]:
from sklearn.svm import SVC
#Los datos son excesivamente grandes para ser procesados en un tiempo considerable, por lo tanto trabajarémos con los primeros mil registros unicamente
kernels = ['linear','rbf','poly']
X_train_bf = X_train_bf.iloc[:10000,:]
X_test_bf =X_test_bf.iloc[:1000,:]
y_train_bf = y_train[:10000]
y_test_bf = y_test[:1000]


scores = []
for k in kernels:
  est = SVC(kernel=k)
  est.fit(X_train_bf,y_train_bf)
  scores.append(accuracy_score(y_test_bf,est.predict(X_test_bf)))


print(scores)


[0.735, 0.608, 0.607]


In [ ]:
##como vemos el kernel que mejor resultado nos dá es el linear entonces usaremos este.

In [ ]:
#listoprint("Random Forest Sin crossval")  
est = SVC(kernel='linear')
est.fit(X_train_bf,y_train_bf)
predict = est.predict(X_test_bf)
acc = accuracy_score(y_test_bf,predict)
print("accuracy %.3f"%acc)

X_ftr = X.iloc[:,[6,8,9,10]];
X_ftr = X_ftr[:1000]
y = y[:1000]

scores = []
stds = []
estimators = range(1,12,1)
print("Cross validation")
est = SVC(kernel='linear')
s = cross_val_score(est, X_ftr, y, cv=KFold(4, shuffle=True), scoring=make_scorer(accuracy_score))
scores.append(np.mean(s))
stds.append(np.std(s))

scores = np.r_[scores]
stds  = np.r_[stds]
print("Desviación máxima: ",np.max(stds))
print("Media máxima: ",np.max(scores))


accuracy 0.735
Cross validation
Desviación máxima:  0.026532998322843223
Media máxima:  0.74


In [ ]:
#Ahora usando pca para obtener unas mejores caracteristicas
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
#Vamos a usar unicamente mil caracteristicas, pues el procesado es bastante costoso temporalmente.
X_ = X.iloc[:,];
X_ = X_[:1000]
y = y[:1000]

X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.1, random_state=42)

nc = range(2,6)
scores = [[],[],[],[]]
scores2 = []
predictions = []

est1 = GaussianNB()
est2 = DecisionTreeClassifier(max_depth = 10)
est3 = RandomForestClassifier(n_estimators = 10,max_depth=10)
est4=  SVC(kernel='linear')

for i in nc:
  pca = PCA(n_components=i)
  pca.fit(X_train)
  xctr = pca.transform(X_train)
  xcts = pca.transform(X_test)

  est1.fit(xctr,y_train)
  scores[0].append(est1.score(xcts,y_test))
  est2.fit(xctr,y_train)
  scores[1].append(est2.score(xcts,y_test))
  est3.fit(xctr,y_train)
  scores[2].append(est3.score(xcts,y_test))
  est4.fit(xctr,y_train)
  scores[3].append(est4.score(xcts,y_test))


print(scores)

print("Resultado medio Gaussian = ",np.mean(scores[0]))
print("Resultado medio DT = ",np.mean(scores[1]))
print("Resultado medio RFC = ",np.mean(scores[2]))
print("Resultado medio SVC = ",np.mean(scores[3]))
    


               Z_LOC       CALI       RMED  ...         BS           ROP      DRHO
623376  -1987.840088  12.281000   2.480526  ...  12.250001    164.857033  0.000314
772679  -2139.433594  14.750621   1.205556  ...  12.270600    164.857033  0.003351
1018900 -2810.497314   8.615782  25.198383  ...   8.500000     22.570070 -0.003648
249406  -1511.126831  13.354495   1.186161  ...  12.250001    164.857033  0.058589
397588  -3105.854736  12.992992  11.690097  ...  12.250001      6.548138 -0.005104
...              ...        ...        ...  ...        ...           ...       ...
452629  -1353.376953  18.321930   1.025489  ...  17.500000      5.157087  0.007006
579931  -1641.733765  17.862848   0.877749  ...  12.270600    164.857033 -0.001427
371509  -2119.414307  12.667389   0.950106  ...  12.250001  32106.001953  0.006029
1144555 -3025.062500   8.835014   4.249170  ...  11.338023    100.308523  0.001736
1147393 -3486.331543   8.489522  13.715576  ...  12.270600    164.857033 -0.000956

[10